# (1)既存モデルの活用：　サイズが大きすぎて使えるか微妙…

In [1]:
%%time
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("./jawiki.doc2vec.dbow300d/jawiki.doc2vec.dbow300d.model")

CPU times: total: 7.14 s
Wall time: 9.74 s


# (2)自作モデルの作成：　精度が出るかわからない…

In [2]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [3]:
import re
import mojimoji
import ipadic
from fugashi import GenericTagger

def clean_text(text):
    text = mojimoji.han_to_zen(text, digit=False, ascii=False)
    text = mojimoji.zen_to_han(text, kana=False)
    text = text.lower()
    for s in ["(57)", "【要約】", "【課題】", "【解決手段】", "\r", "\n", "<br>", "（修正有）","【選択図】"]:
        text = re.sub(s, '', text)
    return text

def tokenizer(text):
    fugger = GenericTagger(ipadic.MECAB_ARGS)
    sentence = []
    for w in fugger(clean_text(text)):
        if("名詞" in w.feature):
            sentence.append(w.surface)
    return sentence

In [4]:
import pandas as pd
# df_input = pd.read_csv('特実_国内文献_B60W4000_B60W5000_20210101-20230109.csv', encoding="utf-8")
df_input = pd.read_csv('data_JPlatPat/特実_国内文献.csv', encoding="utf-8")

In [5]:
df_input.dropna(subset=["要約"], inplace=True)

In [6]:
sentences = []
for text in df_input["要約"].values:
    sentences.append(tokenizer(text))

In [7]:
# words：1文書ずつ、単語に分割したリスト
# tags：文書識別用のタグ情報（リストで指定でき、複数のタグをつけることも可能）
# for文1回あたりの入力イメージは[TaggedDocument([単語1,単語2,単語3,......],[文書タグ])]
trainings = [TaggedDocument(words = sentences[i],tags = [str(i)]) for i in range(len(sentences))]

# 学習
model = Doc2Vec(documents= trainings, dm = 1, vector_size=100, window=8, min_count=10, workers=4, epochs=100)

# モデルの保存("hogehoge"というファイル名で保存)
# model.save("../model/hogehoge.model")

In [8]:
# モデルのロード(既に学習済みのモデルがある場合)
# model = Doc2Vec.load("../model/hogehoge.model")

# 文書タグ0(1つ目の文書)と類似している文書を上から3件(topnで指定)取得し、文書タグと類似度のセットが返ってくる
tag = "0"
print (model.docvecs.most_similar(tag, topn=3))

[('864', 0.7185385227203369), ('27', 0.49857428669929504), ('28', 0.4761306047439575)]


In [9]:
sentences[0]

['車両',
 '車両',
 '状態',
 '車両',
 '周囲',
 '走行',
 '環境',
 'ドライバ',
 '運転',
 '意図',
 '精度',
 '推定',
 '運転',
 '意図',
 '推定',
 '装置',
 '提供',
 '運転',
 '意図',
 '推定',
 '装置',
 'それぞれ',
 '運転',
 '意図',
 '複数',
 '仮想',
 'ドライバ',
 '設定',
 '仮想',
 'ドライバ',
 '運転',
 '操作',
 '量',
 '車両',
 '走行',
 '環境',
 '算出',
 '仮想',
 'ドライバ',
 '運転',
 '操作',
 '量',
 'ドライバ',
 '運転',
 '操作',
 '量',
 '近似',
 '度合',
 'ドライバ',
 '運転',
 '意図',
 '車線',
 '変更',
 '場合',
 '車線',
 '維持',
 '場合',
 '尤',
 '度',
 'それぞれ',
 '算出',
 'ドライバ',
 '運転',
 '意図',
 '判定',
 '際',
 '基準',
 '値',
 '車両',
 '走行',
 '状況',
 '設定',
 '設定',
 '基準',
 '値',
 '車線',
 '変更',
 '尤',
 '度',
 '車線',
 '維持',
 '尤',
 '度',
 'ドライバ',
 '運転',
 '意図',
 '推定',
 '図',
 '1']

In [10]:
sentences[432]

['車両',
 '制御',
 '装置',
 '12',
 '制御',
 '手段',
 '24',
 '車両',
 '10',
 '所定',
 '条件',
 '成立',
 '時',
 '押圧',
 'スイッチ',
 '16',
 '押圧',
 '操作',
 '駆動',
 '源',
 '30',
 '作動',
 '状態',
 '車載',
 '機器',
 '電源',
 '供給',
 '状態',
 '制御',
 '禁止',
 '制御',
 '手段',
 '24',
 '車両',
 '10',
 '異常',
 '状態',
 '発生',
 '検出',
 '場合',
 '前記',
 '所定',
 '条件',
 '成立',
 '押圧',
 'スイッチ',
 '16',
 '押圧',
 '操作',
 '制御',
 '許可']

In [11]:
sentences[864]

['車両',
 'レーン',
 'マーカ',
 '相対',
 '的',
 '関係',
 '検出',
 '場合',
 'ドライバ',
 '運転',
 '意図',
 '安定',
 '的',
 '推定',
 '運転',
 '意図',
 '推定',
 '装置',
 '提供',
 '運転',
 '意図',
 '推定',
 '装置',
 'レーン',
 'マーカ',
 '検出',
 '場合',
 '車両',
 'レーン',
 'マーカ',
 '相対',
 '位置',
 '関係',
 'それぞれ',
 '運転',
 '意図',
 '複数',
 '仮想',
 'ドライバ',
 '設定',
 '運転',
 '者',
 '運転',
 '操作',
 '量',
 '仮想',
 'ドライバ',
 '運転',
 '操作',
 '量',
 '比較',
 '運転',
 '者',
 '運転',
 '意図',
 '推定',
 'レーン',
 'マーカ',
 '検出',
 '場合',
 '車両',
 '先行',
 '車',
 '相対',
 '位置',
 '関係',
 '算出',
 '車両',
 '車線',
 '内',
 '横',
 '位置',
 '複数',
 '仮想',
 'ドライバ',
 '設定',
 'レーン',
 'マーカ',
 '検出',
 '状況',
 '仮想',
 'ドライバ',
 '運転',
 '操作',
 '量',
 '算出',
 '方法',
 '選択',
 '図',
 '1']

In [12]:
#ベクトル化
document_vecs=[]
for i in range(len(sentences)):
    document_vecs.append(model.infer_vector(sentences[i], epochs=30))

In [13]:
input_vec = model.infer_vector(tokenizer("自動運転中の表示"), epochs=30)

In [14]:
#コサイン類似度の計算＋ランキング化
import numpy as np
rank_size = 5
 
v1 = np.linalg.norm(input_vec)
cos_sim = []
for v2 in document_vecs:
    cos_sim.append( np.dot(input_vec,v2)/(v1*np.linalg.norm(v2)) )
doc_sort = np.argsort(np.array(cos_sim))[::-1]
cos_sort = sorted(cos_sim,reverse=True)
 
for i in range(rank_size):
    print(cos_sort[i])
    print(df_input["要約"].values[doc_sort[i]])

0.6725564
(57)【要約】
【課題】運転者自身の運転傾向を、実際の走行中の運転<BR>状況と対応させて視覚的に理解することが可能な運転支<BR>援技術を提供する。
【解決手段】運転支援装置１は走行中に記録された車両<BR>の運転情報に基づき解析した運転傾向の診断結果を運転<BR>者に提示する。運転支援装置１は、運転傾向を解析する<BR>とともに運転情報として記録された実際の走行中の運転<BR>映像１２１と解析した運転傾向の診断結果とを表示装置<BR>２０１に表示させる制御部１０を備える。
【選択図】図１

0.66264313
(57)【要約】
【課題】運転者自身の運転傾向を、実際の走行中の運転<BR>状況と対応させて視覚的に理解することが可能な運転支<BR>援技術を提供する。
【解決手段】運転支援装置１は走行中に記録された車両<BR>の運転情報に基づき解析した運転傾向の診断結果を運転<BR>者に提示する。運転支援装置１は、運転傾向を解析する<BR>とともに運転情報として記録された実際の走行中の運転<BR>映像１２１と解析した運転傾向の診断結果とを表示装置<BR>２０１に表示させる制御部１０を備える。
【選択図】図１

0.61991954
(57)【要約】
【課題】運転者自身の運転傾向を、実際の走行中の運転<BR>状況と対応させて視覚的に理解することが可能な運転支<BR>援技術を提供する。
【解決手段】運転支援装置１は走行中に記録された車両<BR>の運転情報に基づき解析した運転傾向の診断結果を運転<BR>者に提示する。運転支援装置１は、運転傾向を解析する<BR>とともに運転情報として記録された実際の走行中の運転<BR>映像１２１と解析した運転傾向の診断結果とを表示装置<BR>２０１に表示させる制御部１０を備える。
【選択図】図１

0.6123591
(57)【要約】
【課題】通勤時間帯や通学時間帯において、交通安全を<BR>重視した走行をできるようにした自動運転車を提供する<BR>。
【解決手段】手動運転モードと、自律走行を行う自動運<BR>転モードとを有する自動運転車である。通勤時間帯及び<BR>／又は通学時間帯を検知する検知手段と、走行中、駐車<BR>中あるいは停車中に、検知手段で通勤時間帯及び／又は<BR>通学時間帯を検知したことに基づいて、手動